In [1]:
import requests
from bs4 import BeautifulSoup
import re
import queue
import threading

In [49]:
url = 'https://search.51job.com/list/000000,000000,0000,00,9,99,Java,2,1.html?lang=c&stype=&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare='

In [23]:
# 获得已经包装好的ip代理列表
# 修改成queue来存储
# 后面多线程来验证ip
def getIPool():
    proxies = queue.Queue()
    
    with open('ip_pool.txt', 'r') as f:
        ip_pool = f.read()
        
    ip_pool = ip_pool.split('\n')

    # 删除最后一个空值
    del ip_pool[-1]
    
#     proxies = []
    for ip in ip_pool:
        proxy = {'http':ip, 'https':ip}
        proxies.put(proxy)
#         proxies.append(proxy)
        
    return proxies

# # 先拿10个出来
#     for ip in range(10):
#             proxy = {'http':ip_pool[ip], 'https':ip_pool[ip]}
#             proxies.append(proxy)
        
#     return proxies


proxies = getIPool()
proxies

In [24]:
# 获得清理过后的proxies
# 剔除不能连接网页的proxy
# 多线程来清理
# 单线程太慢
def getCleanProxies(proxies, thread_num):    
    url = 'http://www.google.com'
    
    # 有效的ip队列
    ok_proxies = queue.Queue()
    
    
    def check_proxy(thread_proxies):
        
        thread_proxies_size = thread_proxies.qsize()
        
        for i in range(thread_proxies_size):
            thread_proxies_size = thread_proxies.qsize()
            print('当前剩余处理的ip: ', thread_proxies_size)
            if(thread_proxies_size == 0):
                break
                
            proxy = thread_proxies.get()
            try:
                r = requests.get(url, proxies=proxy, timeout=5)
                print(threading.current_thread().name + ' is processing')
                print('r.status_code:', r.status_code)
                if(r.status_code != 200):
                    print('remove proxy:', proxy)
                    # proxyies.remove(proxy)
                else:
                    ok_proxies.put(proxy)

                print('-------------------------------------')

            except requests.Timeout as e:
                print(threading.current_thread().name + ' is processing')
                print(e)
                print('remove proxy:', proxy)
                #proxies.remove(proxy)
                print('-------------------------------------')
            except requests.ConnectionError as e:
                print(threading.current_thread().name + ' is processing')
                print(e)
                print('remove proxy:', proxy)
                #proxies.remove(proxy)
                print('-------------------------------------')
            except requests.InvalidHeader as e:
                print(threading.current_thread().name + ' is processing')
                print(e)
                print('remove proxy:', proxy)
                #proxies.remove(proxy)
                print('-------------------------------------')
      
    
    threads = []
    
    for i in range(thread_num):
        thread = threading.Thread(target=check_proxy, args=(proxies,))
        threads.append(thread)
    
    print('线程开始：')
    for thread in threads:
        print(thread.name + '开始运行')
        thread.start()
    
    # 每次在剩余处理ip为0之后，程序就会停住不动
    # 当当前线程检测到剩余ip为0后，调用join()方法
    print('线程开始停止运行：')
    for thread in threads:
        thread.join()
        print(thread.name + '已停止')
        
                        
            
    
    return ok_proxies

origin_proxies_size = proxies.qsize()
final_proxies = getCleanProxies(proxies, 25)
print(final_proxies.qsize())
error_count = origin_proxies_size - final_proxies.qsize()
print('无效ip: %d, 有效ip: %d' % (error_count, final_proxies.qsize()))

线程开始：
Thread-194开始运行
当前剩余处理的ip: Thread-195开始运行
 300
当前剩余处理的ip: Thread-196开始运行
 299
当前剩余处理的ip: Thread-197开始运行
 298
当前剩余处理的ip: Thread-198开始运行
 297
当前剩余处理的ip: Thread-199开始运行
 296
当前剩余处理的ip: Thread-200开始运行
 295
当前剩余处理的ip: Thread-201开始运行
 294
当前剩余处理的ip: Thread-202开始运行
 293
当前剩余处理的ip: Thread-203开始运行
 292
当前剩余处理的ip: Thread-204开始运行
 291
当前剩余处理的ip: Thread-205开始运行
 290
当前剩余处理的ip: Thread-206开始运行
 289
当前剩余处理的ip: Thread-207开始运行
 288
当前剩余处理的ip: Thread-208开始运行
 287
当前剩余处理的ip: Thread-209开始运行
 286
当前剩余处理的ip: Thread-210开始运行
 285
当前剩余处理的ip: Thread-211开始运行
 284
当前剩余处理的ip: Thread-212开始运行
 283
当前剩余处理的ip: Thread-213开始运行
 282
当前剩余处理的ip: Thread-214开始运行
 281
当前剩余处理的ip: Thread-215开始运行
 280
当前剩余处理的ip: Thread-216开始运行
 279
Thread-198 is processing当前剩余处理的ip: Thread-217开始运行

r.status_code: 200
------------------------------------- 
278当前剩余处理的ip: 
 278
当前剩余处理的ip: Thread-218开始运行
 276
当前剩余处理的ip: 线程开始停止运行：
 275
Thread-207 is processing
r.status_code: 200
-------------------------------------
当前剩余处理的ip:  274
Thread-211 is

Exception in thread Thread-209:
Traceback (most recent call last):
  File "<ipython-input-24-3b1271851e04>", line 24, in check_proxy
    r = requests.get(url, proxies=proxy, timeout=5)
  File "/home/everglow/anaconda3/lib/python3.6/site-packages/requests/api.py", line 72, in get
    return request('get', url, params=params, **kwargs)
  File "/home/everglow/anaconda3/lib/python3.6/site-packages/requests/api.py", line 58, in request
    return session.request(method=method, url=url, **kwargs)
  File "/home/everglow/anaconda3/lib/python3.6/site-packages/requests/sessions.py", line 512, in request
    resp = self.send(prep, **send_kwargs)
  File "/home/everglow/anaconda3/lib/python3.6/site-packages/requests/sessions.py", line 622, in send
    r = adapter.send(request, **kwargs)
  File "/home/everglow/anaconda3/lib/python3.6/site-packages/requests/adapters.py", line 445, in send
    timeout=timeout
  File "/home/everglow/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line

Thread-217 is processing
r.status_code: 200
-------------------------------------
当前剩余处理的ip:  7
Thread-206 is processing
HTTPConnectionPool(host='91.106.94.5', port=8080): Max retries exceeded with url: http://www.google.com/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7f0d607c2550>, 'Connection to 91.106.94.5 timed out. (connect timeout=5)'))
remove proxy: {'http': '91.106.94.5:8080', 'https': '91.106.94.5:8080'}
-------------------------------------
当前剩余处理的ip:  6
Thread-207 is processing
r.status_code: 200
-------------------------------------
当前剩余处理的ip:  5
Thread-218 is processing
r.status_code: 200
-------------------------------------
当前剩余处理的ip:  4
Thread-214 is processing
HTTPConnectionPool(host='45.64.179.117', port=53281): Max retries exceeded with url: http://www.google.com/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7f0d6817c438>, 'Connection to 45.64.179.117 timed out. (connect timeout=5)'))
remove proxy: {

In [30]:
pool = getIPool()
getProxy(pool)

获得的ip_pool大小: 300


{'http': '185.155.32.112:41258', 'https': '185.155.32.112:41258'}

In [33]:
getProxy(pool)

获得的ip_pool大小: 297


{'http': '61.247.189.22:8080', 'https': '61.247.189.22:8080'}